In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim

In [17]:
# Load your data
df = pd.read_csv('../data/20000_games_2023-08-06 15:55:12.860824.csv')

# Select columns you want to use for prediction
selected_columns = ['dealt', 'first_to_play', 'dealer', 'wild_suit',
                    'player0desired', 'player1desired', 'player2desired', 'player3desired',
                    'hand1', 'hand2', 'hand3', 'hand4', 'hand5', 'hand6', 'hand7',
                    'hand8', 'hand9', 'deciding_player']

X = df[selected_columns]
y = df['desired']

# Normalize data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert data to PyTorch tensors
X_train = torch.FloatTensor(X_train)
y_train = torch.FloatTensor(y_train.to_numpy())
X_test = torch.FloatTensor(X_test)
y_test = torch.FloatTensor(y_test.to_numpy())

In [18]:
# Define your model
class Net(nn.Module):
    def __init__(self, n_features):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(n_features, 10)
        self.fc2 = nn.Linear(10, 10)
        self.fc3 = nn.Linear(10, 10)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.log_softmax(self.fc3(x), dim=1)
        return x

# Initialize the model and define loss and optimizer
model = Net(X_train.shape[1])
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [20]:
# Convert your labels to long datatype
y_train = y_train.long()
y_test = y_test.long()

# Train the model
for epoch in range(10000):
    model.train()
    optimizer.zero_grad()
    out = model(X_train)
    loss = criterion(out, y_train)
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print('Train Epoch: {} \tLoss: {:.6f}'.format(epoch, loss.item()))

# Evaluate the model
model.eval()
with torch.no_grad():
    output = model(X_test)
    predicted = torch.argmax(output, dim=1)
    correct = (predicted == y_test).sum().item()
    accuracy = correct / len(y_test)
    print('Test accuracy: {:.2f}%'.format(accuracy * 100))

Train Epoch: 0 	Loss: 1.091297
Train Epoch: 10 	Loss: 1.090370
Train Epoch: 20 	Loss: 1.089465
Train Epoch: 30 	Loss: 1.088581
Train Epoch: 40 	Loss: 1.087717
Train Epoch: 50 	Loss: 1.086872
Train Epoch: 60 	Loss: 1.086046
Train Epoch: 70 	Loss: 1.085236
Train Epoch: 80 	Loss: 1.084443
Train Epoch: 90 	Loss: 1.083666
Train Epoch: 100 	Loss: 1.082903
Train Epoch: 110 	Loss: 1.082155
Train Epoch: 120 	Loss: 1.081421
Train Epoch: 130 	Loss: 1.080700
Train Epoch: 140 	Loss: 1.079992
Train Epoch: 150 	Loss: 1.079295
Train Epoch: 160 	Loss: 1.078611
Train Epoch: 170 	Loss: 1.077937
Train Epoch: 180 	Loss: 1.077273
Train Epoch: 190 	Loss: 1.076621
Train Epoch: 200 	Loss: 1.075978
Train Epoch: 210 	Loss: 1.075345
Train Epoch: 220 	Loss: 1.074721
Train Epoch: 230 	Loss: 1.074107
Train Epoch: 240 	Loss: 1.073501
Train Epoch: 250 	Loss: 1.072902
Train Epoch: 260 	Loss: 1.072312
Train Epoch: 270 	Loss: 1.071730
Train Epoch: 280 	Loss: 1.071155
Train Epoch: 290 	Loss: 1.070588
Train Epoch: 300 	Los